In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

df = pd.read_table("birthweight.dat", sep="\s+")
print(df.head(2))

x_vals = df[['AGE', 'LWT', 'RACE', 'SMOKE', 'PTL', 'HT', 'UI']]

y_vals = df['BWT']

from sklearn.model_selection import train_test_split

x_vals_train,x_vals_test,y_vals_train,y_vals_test = train_test_split(x_vals,y_vals,test_size=0.2,random_state=42)

# Record training column max and min for scaling of non-training data
train_max = np.max(x_vals_train, axis=0)
train_min = np.min(x_vals_train, axis=0)

# Normalize by column (min-max norm to be between 0 and 1)
def normalize_cols(mat, max_vals, min_vals):
    return (mat - min_vals) / (max_vals - min_vals)

x_vals_train = np.nan_to_num(normalize_cols(x_vals_train, train_max, train_min))
x_vals_test = np.nan_to_num(normalize_cols(x_vals_test, train_max, train_min))

x_vals_train =tf.cast(tf.constant(x_vals_train),tf.float32)

y_vals = tf.expand_dims(tf.cast(tf.constant(y_vals_train),tf.float32),1)


   LOW  AGE  LWT  RACE  SMOKE  PTL  HT  UI   BWT
0    1   28  113     1      1    1   0   1   709
1    1   29  130     0      0    0   0   1  1021


In [3]:
# Define Variable Functions (weights and bias)
def init_weight(shape, st_dev):
    weight = tf.Variable(tf.random.normal(shape, stddev=st_dev))
    return(weight)

def init_bias(shape, st_dev):
    bias = tf.Variable(tf.random.normal(shape, stddev=st_dev))
    return(bias)
    
# Create a fully connected layer:
def fully_connected(input_layer, weights, biases):
    layer = tf.add(tf.matmul(input_layer, weights), biases)
    return(tf.nn.relu(layer))


In [4]:
# x_vals_train : 7 Inputs

#--------Create the first layer (50 hidden nodes)--------
weight_1 = init_weight(shape=[7,25],st_dev=10.0)
bias_1 = init_bias(shape=[25],st_dev=10.0)
layer_1 = fully_connected(x_vals_train,weight_1,bias_1)  # shape= (151,25)


#--------Create second layer (25 hidden nodes)--------
weight_2 = init_weight(shape=[25,10],st_dev=10.0)
bias_2 = init_bias(shape=[10],st_dev=10.0)
layer_2 = fully_connected(layer_1,weight_2,bias_2)  # shape = (151,10)

#--------Create third layer (5 hidden nodes)--------
weight_3 = init_weight(shape=[10,3],st_dev=10.0)
bias_3 = init_bias(shape=[3],st_dev=10.0)
layer_3 = fully_connected(layer_2,weight_3,bias_3) # shape = (151,3)

#--------Create output layer (1 output value)--------
weight_4 = init_weight(shape=[3,1],st_dev=10.0)
bias_4 = init_bias(shape=[1],st_dev=10.0)
# final_output = fully_connected(layer_3,weight_4,bias_4)  # shape = (151,1)


loss = []

epochs = []
optimizer = tf.optimizers.Adam(0.025)


for i in range(100):
    with tf.GradientTape() as t:
        final_output = fully_connected(layer_3,weight_4,bias_4)  # shape = (151,1)
        curr_loss = tf.reduce_mean(tf.abs(y_vals - final_output))
        
    grads = t.gradient(curr_loss,[weight_4,bias_4])
    optimizer.apply_gradients(zip(grads,[weight_4,bias_4]))
    
    loss.append(curr_loss.numpy())
    epochs.append(i)

import pandas as pd

pd.DataFrame(data = {"Iteration":epochs,"Loss":loss})

,Iteration,Loss
0,0,80251.023438
1,1,80130.257812
2,2,80009.484375
3,3,79888.718750
4,4,79767.937500
...,...,...
95,95,68983.117188
96,96,68869.359375
97,97,68755.507812
98,98,68640.390625
